## WFP-01-02-03 Data transformation application - Sentinel-1 backscatter Sigma-0 all polarization

This application is part of the usage of Sentinel-1 complex and backscatter data for detection and quantification of food security related (natural) hazards and land cover/ land use change

### <a name="quicklink">Quick link

* [Objective](#Objective)
* [Data](#Data)
* [Service Definition](#Service-Definition)
* [Parameter Definition](#Parameter-Definition)
* [Runtime Parameter Definition](#Runtime-Parameter-Definition)
* [Workflow](#Workflow)
* [License](#License)

### Objective 

Process Sentinel-1 GRD with:

* Application of orbit file (if available, but should not hold the processing of the product)
* Border noise removal (if necessary)
* Calibration
* (Multi-temporal) Speckle filtering 
* Terrain correction
* Conversion to dB

### Data 

SENTINEL data products are made available systematically and free of charge to all data users including the general public, scientific and commercial users. Radar data will be delivered within an hour of reception for Near Real-Time (NRT) emergency response, within three hours for NRT priority areas and within 24 hours for systematically archived data.

All data products are distributed in the SENTINEL Standard Archive Format for Europe (SAFE) format.

Sentinel-1 data products are available in single polarisation (VV or HH) for Wave mode and dual polarisation (VV+VH or HH+HV) and single polarisation (HH or VV) for SM, IW and EW modes.

Level-1 Ground Range Detected (GRD) products consist of focused SAR data that has been detected, multi-looked and projected to ground range using an Earth ellipsoid model. Phase information is lost. The resulting product has approximately square resolution pixels and square pixel spacing with reduced speckle at the cost of reduced geometric resolution.

GRD products can be in one of three resolutions:

* Full Resolution (FR)
* High Resolution (HR)
* Medium Resolution (MR).

The resolution is dependent upon the amount of multi-looking performed. Level-1 GRD products are available in MR and HR for IW and EW modes, MR for WV mode and MR, HR and FR for SM mode.

### Service Definition 

In [1]:
service = dict([('title', 'WFP-01-02-03 Sentinel-1 backscatter all polarization'),
                ('abstract', 'WFP-01-02-03 Data transformation application - Sentinel-1 backscatter all polarization'),
                ('id', 'wfp-01-02-03')])

### Parameter Definition 

**Speckle-Filter filterSizeX**

In [2]:
filterSizeX = dict([('id', 'filterSizeX'),
               ('value', '5'),
               ('title', 'Speckle-Filter filterSizeX'),
               ('abstract', 'Set the Speckle-Filter filterSizeX (defaults to 5)')])

**Speckle-Filter filterSizeY**

In [3]:
filterSizeY = dict([('id', 'filterSizeY'),
               ('value', '5'),
               ('title', 'Speckle-Filter filterSizeY'),
               ('abstract', 'Set the Speckle-Filter filterSizeY (defaults to 5)')])

**Polarisation mode**

The Sentinel-1 C-band SAR instruments supports operation in single polarisation (HH or VV) and dual polarisation (HH+HV or VV+VH), implemented through one transmit chain (switchable to H or V) and two parallel receive chains for H and V polarisation.

Let's keep the polarisation parameter empty, this way GPT will process all bands.

In [4]:
#polarisation = dict([('id', 'polarisation'),
#               ('value', 'VV'),
#               ('title', 'Sentinel-1 polarisation (VV or HH)'),
#               ('abstract', 'Sentinel-1 polarisation (VV or HH)')])

**Area of interest**

Define the area of interest using a polygon in Well-Known-Text format

In [5]:
wkt = dict([('id', 'wkt'),
            ('value', 'POLYGON((-5.5 17.26, -1.08 17.26, -1.08 13.5, -5.5 13.5, -5.5 17.26))'),
            ('title', 'Area of interest in WKT'),
            ('abstract', 'Area of interest using a polygon in Well-Known-Text format')])

### Runtime parameter definition

**Input identifier**

This is the Sentinel-1 GRD product identifier, e.g. **S1A_IW_GRDH_1SDV_20171210T182024_20171210T182049_019644_021603_0A33**

In [6]:
input_identifier = 'S1A_IW_GRDH_1SDV_20171210T182024_20171210T182049_019644_021603_0A33'

**Input reference**

This is the Sentinel-1 GRD product catalogue reference

In [7]:
input_reference = 'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_GRDH_1SDV_20171210T182024_20171210T182049_019644_021603_0A33' 

**Data path**

This path defines where the data is staged-in. 

In [8]:
data_path = '/workspace/data'

### Workflow

#### Import the packages required for processing the Sentinel-1 backscatter

In [9]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")
import os
import sys
import glob
sys.path.append('/opt/anaconda/bin/')

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors

from snappy import jpy
from snappy import ProductIO
from snappy import GPF
from snappy import HashMap

import gc

#### Read the product

In [10]:
s1meta = "manifest.safe"

s1prd = os.path.join(data_path, input_identifier, input_identifier + '.SAFE', s1meta)

reader = ProductIO.getProductReader("SENTINEL-1")
product = reader.readProductNodes(s1prd, None)

#### ThermalNoiseRemoval step

In [11]:
HashMap = jpy.get_type('java.util.HashMap')    
GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()

parameters = HashMap()

#parameters.put('selectedPolarisations', polarisation['value'])
parameters.put('removeThermalNoise', 'true')
parameters.put('reIntroduceThermalNoise', 'false')

thermal_noise_removal = GPF.createProduct('ThermalNoiseRemoval', parameters, product)

#### Apply-Orbit-File

In [12]:
parameters = HashMap()

parameters.put('orbitType', 'Sentinel Precise (Auto Download)')
parameters.put('polyDegree', '3')
parameters.put('continueOnFail', 'false')

apply_orbit_file = GPF.createProduct('Apply-Orbit-File', parameters, thermal_noise_removal)

#### Calibration

In [13]:
parameters = HashMap()

parameters.put('auxFile', 'Product Auxiliary File')
parameters.put('outputImageInComplex', 'false')
parameters.put('outputImageScaleInDb', 'false')
parameters.put('createGammaBand', 'false')
parameters.put('createBetaBand', 'true')
#parameters.put('selectedPolarisations', polarisation['value'])
parameters.put('outputSigmaBand', 'false')
parameters.put('outputGammaBand', 'false')
parameters.put('outputBetaBand', 'true')

calibration = GPF.createProduct('Calibration', parameters, apply_orbit_file)

#### Speckle-Filter

In [14]:
parameters = HashMap()

#parameters.put('sourceBands', 'Beta0_%s' % (polarisation['value']))
parameters.put('filter', 'Lee')
parameters.put('filterSizeX', filterSizeX['value'])
parameters.put('filterSizeY', filterSizeY['value'])
parameters.put('dampingFactor', '2')
parameters.put('estimateENL', 'true')
parameters.put('enl', '1.0')
parameters.put('numLooksStr', '1')
parameters.put('targetWindowSizeStr', '3x3')
parameters.put('sigmaStr', '0.9')
parameters.put('anSize', '50')

speckle_filter = GPF.createProduct('Speckle-Filter', parameters, calibration)

#### Terrain-Correction

In [15]:
parameters = HashMap()

#parameters.put('sourceBands', 'Beta0_%s' % (polarisation['value']))
parameters.put('demName', 'SRTM 3Sec')
parameters.put('externalDEMFile', '')
parameters.put('externalDEMNoDataValue', '0.0')
parameters.put('externalDEMApplyEGM', 'true')
parameters.put('demResamplingMethod', 'BILINEAR_INTERPOLATION')
parameters.put('imgResamplingMethod', 'BILINEAR_INTERPOLATION')
parameters.put('pixelSpacingInMeter', '10.0')
#parameters.put('pixelSpacingInDegree', '8.983152841195215E-5')
parameters.put('mapProjection', 'AUTO:42001')
parameters.put('nodataValueAtSea', 'true')
parameters.put('saveDEM', 'false')
parameters.put('saveLatLon', 'false')
parameters.put('saveIncidenceAngleFromEllipsoid', 'false')
parameters.put('saveProjectedLocalIncidenceAngle', 'false')
parameters.put('saveSelectedSourceBand', 'true')
parameters.put('outputComplex', 'false')
parameters.put('applyRadiometricNormalization', 'false')
parameters.put('saveSigmaNought', 'false')
parameters.put('saveGammaNought', 'false')
parameters.put('saveBetaNought', 'false')
parameters.put('incidenceAngleForSigma0', 'Use projected local incidence angle from DEM')
parameters.put('incidenceAngleForGamma0', 'Use projected local incidence angle from DEM')
parameters.put('auxFile', 'Latest Auxiliary File')

terrain_correction = GPF.createProduct('Terrain-Correction', parameters, speckle_filter)

#### Linear to dB

In [16]:
parameters = HashMap()

lineartodb = GPF.createProduct('linearToFromdB', parameters, terrain_correction)

#### Subset

In [17]:
#parameters = HashMap()

#parameters.put('sourceBands', 'Beta0_%s' % (polarisation['value']))
##parameters.put('region', '')
#parameters.put('geoRegion', wkt['value'])
#parameters.put('subSamplingX', '1')
#parameters.put('subSamplingY', '1')
#parameters.put('fullSwath', 'false')
#parameters.put('tiePointGridNames', '')
#parameters.put('copyMetadata', 'true')

#subset = GPF.createProduct('Subset', parameters, terrain_correction)

#### Garbage collector

In [18]:
thermal_noise_removal = None
apply_orbit_file = None
calibration = None
speckle_filter = None
terrain_correction = None

gc.collect()

17

#### Save the result

In [ ]:
output_name = '%s_Beta0_%s.tif' % (input_identifier, 'all_bands')

In [ ]:
ProductIO.writeProduct(lineartodb, output_name, 'GeoTIFF-BigTIFF')

### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.